In [1]:
import pandas as pd
import numpy as np
import cfscrape
from bs4 import BeautifulSoup
from lxml import etree
import re
import time
from tqdm import tqdm
from ipykernel import kernelapp as app

pd.options.mode.chained_assignment = None

In [2]:
links = pd.read_excel('page_items_output_2.xlsx')
links = links.drop(['Unnamed: 0','Unnamed: 3','Unnamed: 4'], axis=1)
links = links.drop_duplicates(ignore_index=True)

In [3]:
links

,Link,Região
0,-apartamento-3-quartos-com-elevador-funcionari...,mg+belo-horizonte
1,-apartamento-2-quartos-com-piscina-lourdes-bel...,mg+belo-horizonte
2,-apartamento-4-quartos-com-academia-serra-belo...,mg+belo-horizonte
3,-apartamento-2-quartos-com-piscina-santo-agost...,mg+belo-horizonte
4,-apartamento-3-quartos-cidade-nova-belo-horizo...,mg+belo-horizonte
...,...,...
25709,-apartamento-3-quartos-com-interfone-jardim-ca...,es+vitoria
25710,-apartamento-3-quartos-com-ar-condicionado-san...,es+vitoria
25711,-apartamento-1-quarto-com-churrasqueira-praia-...,es+vitoria
25712,-apartamento-3-quartos-com-jardim-praia-do-can...,es+vitoria


In [4]:
links.shape

(25714, 2)

In [5]:
# Função para buscar os dados dentro das páginas.
def buscar_dados():
    
    #----------------------------------------------------------------------------------------------------- Valor Aluguel
    se_venda_ou_locação = soup.find(class_='main-prices hybrid-business')

    if se_venda_ou_locação:
        aluguel = html_page.xpath('//*[@id="app"]/section/article[1]/div/div[1]/div[2]/div/ul[1]/li[2]/strong/text()')
        aluguel = aluguel[0]

        s_aluguel = str(aluguel)
        pattern = re.compile(r'\s+$')    
        s_aluguel = re.sub(pattern, '', s_aluguel)
        pattern = re.compile(r'^\s+')
        aluguel = re.sub(pattern, '', s_aluguel)
    else:
        aluguel = html_page.xpath('//*[@id="app"]/section/article[1]/div/div[1]/div[2]/div/ul[1]/li/strong/text()')
        aluguel = aluguel[0]

        s_aluguel = str(aluguel)
        pattern = re.compile(r'\s+$')    
        s_aluguel = re.sub(pattern, '', s_aluguel)
        pattern = re.compile(r'^\s+')
        aluguel = re.sub(pattern, '', s_aluguel)

    imoveis_Valor_aluguel.append(int(aluguel.replace("R$ ", "").replace(".", "")))

    #-------------------------------------------------------------------------------------------------- Valor Condominio

    condominio = html_page.xpath('//*[@id="app"]/section/article[1]/div/div[1]/div[2]/div/ul[2]/li[1]/span/text()')
    condominio = condominio[0]

    if condominio != "não informado":
        s_condominio = str(condominio)
        pattern = re.compile(r'\s+$')    
        s_condominio = re.sub(pattern, '', s_condominio)
        pattern = re.compile(r'^\s+')
        condominio = re.sub(pattern, '', s_condominio)
        condominio = condominio.replace("R$ ", "").replace(".", "")
    else:
        condominio = 0

    imoveis_Valor_condomínio.append(int(condominio))

    #-------------------------------------------------------------------------------------------------------- Valor IPTU

    iptu = html_page.xpath('//*[@id="app"]/section/article[1]/div/div[1]/div[2]/div/ul[2]/li[2]/span/text()')
    iptu = iptu[0]

    if iptu != "não informado":
        s_iptu = str(iptu)
        pattern = re.compile(r'\s+$')    
        s_iptu = re.sub(pattern, '', s_iptu)
        pattern = re.compile(r'^\s+')
        iptu = re.sub(pattern, '', s_iptu)
        iptu = iptu.replace("R$ ", "").replace(".", "")
    else:
        iptu = 0

    imoveis_Valor_IPTU.append(int(iptu))

    #-------------------------------------------------------------------------------------------------------------- Área

    area_imovel = soup.find(class_='feature__item text-regular js-areas')
    
    if area_imovel:
        s_area_imovel = area_imovel.text
        pattern = re.compile(r'\s+$')    
        s_area_imovel = re.sub(pattern, '', s_area_imovel)
        pattern = re.compile(r'^\s+')
        area_imovel = re.sub(pattern, '', s_area_imovel)
        area_imovel = area_imovel.replace(" m²", "")
    else:
        area_imovel = 0
    
    imoveis_Area.append(area_imovel)

    #----------------------------------------------------------------------------------------------------------- Quartos
    
    qt_quartos = soup.find(class_='feature__item text-regular js-bedrooms')
    
    if qt_quartos: 
        s_qt_quartos = qt_quartos.text
        pattern = re.compile(r'\s+$')    
        s_qt_quartos = re.sub(pattern, '', s_qt_quartos)
        pattern = re.compile(r'^\s+')
        qt_quartos = re.sub(pattern, '', s_qt_quartos)
        
        if qt_quartos == "1 quarto":
            qt_quartos = qt_quartos.replace(" quarto", "")
        else:
            qt_quartos = qt_quartos.replace(" quartos", "")
    else:
        qt_quartos = 0
        
    imoveis_Quartos.append(qt_quartos)
    
    #------------------------------------------------------------------------------------------------------------ Vagas

    qt_vagas = soup.find(class_='feature__item text-regular js-parking-spaces')
    
    if qt_vagas:
        s_qt_vagas = qt_vagas.text
        pattern = re.compile(r'\s+$')    
        s_qt_vagas = re.sub(pattern, '', s_qt_vagas)
        pattern = re.compile(r'^\s+')
        qt_vagas = re.sub(pattern, '', s_qt_vagas)
        
        if qt_vagas == "1 vaga":
            qt_vagas = qt_vagas.replace(" vaga", "")
        else:
            qt_vagas = qt_vagas.replace(" vagas", "")
        
    else:
        qt_vagas = 0
        
    imoveis_Vagas.append(qt_vagas)
        
    #-------------------------------------------------------------------------------------------------------- Banheiros

    qt_banheiros = soup.find(class_='feature__item text-regular js-bathrooms')
    
    if qt_banheiros:
        s_qt_banheiros = qt_banheiros.text
        pattern = re.compile(r'\s+$')    
        s_qt_banheiros = re.sub(pattern, '', s_qt_banheiros)
        pattern = re.compile(r'^\s+')
        qt_banheiros = re.sub(pattern, '', s_qt_banheiros)
        
        if qt_banheiros == "1 banheiro":
            qt_banheiros = qt_banheiros.replace(" banheiro", "")
        else:
            qt_banheiros = qt_banheiros.replace(" banheiros", "")
        
    else:
        qt_banheiros = 0

    imoveis_Banheiros.append(qt_banheiros)

    #------------------------------------------------------------------------------------------- Características físicas

    qt_item_caracteristica = soup.find(class_='amenities__list')
    string_xpath = '//*[@id="app"]/section/div[2]/article/div[1]/aside[1]/div/div/ul/li[{n}]/text()'
    caracteristicas = []
    
    if qt_item_caracteristica:
        for x in range(1,len(qt_item_caracteristica)+1):

            xpath_full = string_xpath.format(n = x)
            carac = html_page.xpath(xpath_full)[0]

            s = str(carac)
            pattern = re.compile(r'\s+$')    
            s = re.sub(pattern, '', s)
            pattern = re.compile(r'^\s+')
            s = re.sub(pattern, '', s)

            caracteristicas.append(s)
    else:
        caracteristicas = None

    #separator = '/'
    #array_p_separar = caracteristicas
    #string_join = [separator.join(array_p_separar)]
    #caracteristicas = string_join[0]

    imoveis_Caracteristicas.append(caracteristicas)

    #--------------------------------------------------------------------------------------------------------- Descrições

    Descrição_imovel = soup.find(class_='amenities__description text-regular text-margin-zero').text
    imoveis_Descricao.append(Descrição_imovel)

In [96]:
imoveis_Link = []
imoveis_Estado = []
imoveis_Cidade = []
imoveis_Valor_aluguel = []
imoveis_Valor_condomínio = []
imoveis_Valor_IPTU = []
imoveis_Area = []
imoveis_Quartos = []
imoveis_Vagas = []
imoveis_Banheiros = []
imoveis_Caracteristicas = []
imoveis_Descricao = []#25700

In [100]:
imoveis = pd.DataFrame()

tamanho_slicing = len(imoveis_backup)
x_slincing = tamanho_slicing
y_slincing = tamanho_slicing + 50

pagina = "https://www.zapimoveis.com.br/imovel/aluguel{link}"

x = 1
while y_slincing <= 25700:
    time.sleep(x)
    for index,page in enumerate(tqdm(links['Link'][x_slincing:y_slincing], "Buscando link")):
        
        scraper = cfscrape.create_scraper()
        url = pagina.format(link = page)
        result = scraper.get(url)
        soup = BeautifulSoup(result.content, 'html.parser')
        html_page = etree.HTML(str(soup))

        if result.status_code == 200:
            imoveis_Link.append(page)
            imoveis_Estado.append(links['Região'][index].split("+")[0].upper())
            imoveis_Cidade.append(links['Região'][index].split("+")[1].replace("-", " "))
            buscar_dados()
            
            imoveis["Link"] = imoveis_Link
            imoveis["Estado"] = imoveis_Estado
            imoveis["Cidade"] = imoveis_Cidade
            imoveis["Valor_aluguel"] = imoveis_Valor_aluguel
            imoveis["Valor_condomínio"] = imoveis_Valor_condomínio
            imoveis["Valor_iptu"] = imoveis_Valor_IPTU
            imoveis["Área_total"] = imoveis_Area
            imoveis["Qt_quartos"] = imoveis_Quartos
            imoveis["Qt_vagas"] = imoveis_Vagas
            imoveis["Qt_banheiros"] = imoveis_Banheiros
            imoveis["Características"] = imoveis_Caracteristicas
            imoveis["Descrição"] = imoveis_Descricao
            
            imoveis.to_excel('imoveis2.xlsx')
            imoveis = pd.DataFrame()
            
        elif result.status_code == 404:
            imoveis_Link.append(page)
            imoveis_Estado.append(links['Região'][index].split("+")[0].upper())
            imoveis_Cidade.append(links['Região'][index].split("+")[1].replace("-", " "))
            imoveis_Valor_aluguel.append(None)
            imoveis_Valor_condomínio.append(None)
            imoveis_Valor_IPTU.append(None)
            imoveis_Area.append(None)
            imoveis_Quartos.append(None)
            imoveis_Vagas.append(None)
            imoveis_Banheiros.append(None)
            imoveis_Caracteristicas.append(None)
            imoveis_Descricao.append(None)
            
            imoveis["Link"] = imoveis_Link
            imoveis["Estado"] = imoveis_Estado
            imoveis["Cidade"] = imoveis_Cidade
            imoveis["Valor_aluguel"] = imoveis_Valor_aluguel
            imoveis["Valor_condomínio"] = imoveis_Valor_condomínio
            imoveis["Valor_iptu"] = imoveis_Valor_IPTU
            imoveis["Área_total"] = imoveis_Area
            imoveis["Qt_quartos"] = imoveis_Quartos
            imoveis["Qt_vagas"] = imoveis_Vagas
            imoveis["Qt_banheiros"] = imoveis_Banheiros
            imoveis["Características"] = imoveis_Caracteristicas
            imoveis["Descrição"] = imoveis_Descricao
            
            imoveis.to_excel('imoveis2.xlsx')
            imoveis = pd.DataFrame()
            
        elif result.status_code == 429:
            print("Erro code 429!")
            print("Parou: ",index)
            y_slincing += 30000
            break


        time.sleep(1)

    x_slincing += 50
    y_slincing += 50
    x = 150

Buscando link: 100%|███████████████████████████████████████████████████████████████████| 50/50 [03:27<00:00,  4.14s/it]


In [12]:
# Carregando os arquivos do excel que estão separados, pois eles foram extraídos em 2 partes
imoveis = pd.read_excel('imoveis.xlsx')
imoveis = imoveis.drop(['Unnamed: 0'], axis=1)

imoveis2 = pd.read_excel('imoveis2.xlsx')
imoveis2 = imoveis2.drop(['Unnamed: 0'], axis=1)

links = pd.read_excel('page_items_output_2.xlsx')
links = links.drop(['Unnamed: 0','Unnamed: 4'], axis=1)
links = links.drop_duplicates(ignore_index=True)

In [70]:
# Fazendo um merge para unir a base de dados com as regiões corretas
imoveis = imoveis.merge(links, left_on='Link', right_on='Link',suffixes=('_left', '_right'))
imoveis2 = imoveis2.merge(links, left_on='Link', right_on='Link',suffixes=('_left', '_right'))

In [79]:
# Organizando a sequência das colunas
imoveis = imoveis[['Link', 'Estado_new', 'Cidade_new', 'Valor_aluguel', 'Valor_condomínio',
       'Valor_iptu', 'Área_total', 'Qt_quartos', 'Qt_vagas', 'Qt_banheiros',
       'Características', 'Descrição']]

imoveis2 = imoveis2[['Link', 'Estado_new', 'Cidade_new', 'Valor_aluguel', 'Valor_condomínio',
       'Valor_iptu', 'Área_total', 'Qt_quartos', 'Qt_vagas', 'Qt_banheiros',
       'Características', 'Descrição']]

In [90]:
# Renomeando as colunas estado e cidade
imoveis.columns = ['Link', 'Estado', 'Cidade', 'Valor_aluguel', 'Valor_condomínio',
       'Valor_iptu', 'Área_total', 'Qt_quartos', 'Qt_vagas', 'Qt_banheiros',
       'Características', 'Descrição']

imoveis2.columns = ['Link', 'Estado', 'Cidade', 'Valor_aluguel', 'Valor_condomínio',
       'Valor_iptu', 'Área_total', 'Qt_quartos', 'Qt_vagas', 'Qt_banheiros',
       'Características', 'Descrição']

In [95]:
# Unindo os dois dataframe em um só
imoveis_full = pd.concat([imoveis,imoveis2])

In [112]:
# Modificando o nome da cidade de "belo-horizonte" para "belo horizonte"
imoveis_full['Cidade'] = np.where(imoveis_full['Cidade'] == "belo-horizonte", "belo horizonte", imoveis_full['Cidade'])

In [113]:
imoveis_full.shape

(25695, 12)

In [125]:
# Resetando o index
imoveis_full = imoveis_full.reset_index(drop=True)

In [128]:
pd.set_option('display.max_colwidth', 50)
imoveis_full

,Link,Estado,Cidade,Valor_aluguel,Valor_condomínio,Valor_iptu,Área_total,Qt_quartos,Qt_vagas,Qt_banheiros,Características,Descrição
0,-apartamento-3-quartos-com-elevador-funcionari...,mg,belo horizonte,3250.0,700.0,0.0,75.0,3.0,2.0,2.0,['Elevador'],"Codigo do Imóvel: 4058 - NOVO, ÓTIMA LOCALIAZA..."
1,-apartamento-2-quartos-com-piscina-lourdes-bel...,mg,belo horizonte,4800.0,1100.0,0.0,70.0,2.0,2.0,1.0,"['Área de serviço', 'Armário na cozinha', 'Ele...",Codigo do Imóvel: 4036 - PRÉDIO REVESTIDO EM G...
2,-apartamento-4-quartos-com-academia-serra-belo...,mg,belo horizonte,7300.0,3300.0,1100.0,190.0,4.0,4.0,5.0,"['Academia', 'Interfone', 'Jardim', 'Playgroun...",Excelente localização próximo ao Minas Tênis C...
3,-apartamento-2-quartos-com-piscina-santo-agost...,mg,belo horizonte,3100.0,856.0,290.0,60.0,2.0,2.0,2.0,"['Academia', 'Churrasqueira', 'Condomínio fech...",Prédio moderno; apartamento nunca habitado; 2 ...
4,-apartamento-3-quartos-cidade-nova-belo-horizo...,mg,belo horizonte,1500.0,280.0,160.0,110.0,3.0,2.0,3.0,"['Área de serviço', 'Interfone', 'Salão de fes...",Alugo apartamento amplo com duas salas separad...
...,...,...,...,...,...,...,...,...,...,...,...,...
25690,-apartamento-4-quartos-com-interfone-barro-ver...,es,vitoria,11000.0,1500.0,0.0,344.0,4.0,4.0,6.0,"['Área de serviço', 'Cabeamento estruturado', ...","Oportunidade:apartamento 4 suítes, frente, sol..."
25691,-apartamento-4-quartos-com-interfone-praia-do-...,es,vitoria,6000.0,1400.0,0.0,175.0,4.0,2.0,4.0,"['Área de serviço', 'Cabeamento estruturado', ...",Apartamento 4 quartos 1 suite na Praia do Cant...
25692,-apartamento-4-quartos-mobiliado-praia-do-cant...,es,vitoria,9015.0,2300.0,0.0,243.0,4.0,2.0,5.0,"['Aquecimento', 'Ar-condicionado', 'Área de se...","Apartamento Praia do Canto frente para o mar, ..."
25693,-apartamento-4-quartos-mobiliado-praia-do-cant...,es,vitoria,25000.0,0.0,0.0,400.0,4.0,5.0,5.0,"['Ar-condicionado', 'Cozinha', 'Elevador', 'Es...","Luxuoso apartamento na Praia do Canto, disponí..."


In [129]:
imoveis_full.to_excel('imoveis_full.xlsx')